# Testing of features in polysaccharide2

In [1]:
# Supressing annoying warnings (!must be done first!)
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

# Logging
from polysaccharide2.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    # force=True
)
LOGGER = logging.Logger(__name__)

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np

# Logging
from rich.progress import Progress, track
import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polysaccharide2 as ps2
from polysaccharide2.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/timber/miniconda3/envs/openff-pdb-pr/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


# OpenFF parameterization testing

## Testing load using from_pdb and monomers

In [ ]:
from polysaccharide2.openfftools import TKREGS
from polysaccharide2.openfftools import topology
from polysaccharide2.residues.partition import partition
from polysaccharide2.monomers.repr import MonomerGroup

# pdb_dir  = Path('polymer_examples/compatible_pdbs/simple_polymers')
# mono_dir = Path('polymer_examples/monomer_generation/json_files/')

pdb_sub = 'simple_polymers'
pdb_dir  = Path(f'pdb_test_cleaned/pdbs/{pdb_sub}')
mono_dir = Path(f'pdb_test_cleaned/monos/{pdb_sub}')

mol_name = 'polyvinylchloride'
# mol_name = 'PEO_PLGA'
# mol_name = 'paam_modified'
# pdb_sub = 'proteins'
# mol_name = '6cww'

pdb = pdb_dir / f'{mol_name}.pdb'
mono = mono_dir / f'{mol_name}.json'
assert(pdb.exists())
assert(mono.exists())

monogrp = MonomerGroup.from_file(mono)
rdmol = Chem.MolFromPDBFile(str(pdb))
offtop = Topology.from_pdb(pdb, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit'])
was_partitioned = partition(offtop)
print(was_partitioned)

# assign properties to Molecule
offmol =topology.get_largest_offmol(offtop)
offmol.name = mol_name
offmol.properties['solvent'] = None
offmol.properties['charge_method'] = None

# save partitioned Topology
sdf_dir = Path('sdf_test')
sdf_dir.mkdir(exist_ok=True)
mol_path = sdf_dir / f'{mol_name}.sdf'
topology.topology_to_sdf(mol_path, offtop=offtop, toolkit_registry=TKREGS['The RDKit'])

## Partial charge assignment

In [ ]:
from polysaccharide2.openfftools import TKREGS
from polysaccharide2.openfftools import topology
from polysaccharide2.residues.rescharge import application, calculation

base_charge_method = 'AM1-BCC-ELF10'

# assign charges with default methods
charged_mols = {}
for charge_method, ChargerType in application.MolCharger.subclass_registry.items():
    chgr = ChargerType()
    cmol = charged_mols[charge_method] = chgr.charge_molecule(offmol, in_place=False)

# generate library charges and charge by residue
res_chg = calculation.compute_residue_charges(charged_mols[base_charge_method], monogrp)
res_chg.to_file(sdf_dir / f'{mol_name}_residue_charges.json')

offmol_avg = application.apply_residue_charges(offmol, res_chg, in_place=False)
offmol_avg.properties['charge_method'] = 'RCT-averaged'
charged_mols['RCT-averaged'] = offmol_avg

# saving charged molecules to SDF files
for charge_method, cmol in charged_mols.items():
    topology.topology_to_sdf(sdf_dir / f'{mol_name}_{charge_method}.sdf', cmol.to_topology())

## Testing building and RCT

In [ ]:
from polysaccharide2.openfftools.pcharge import MolCharger
from polysaccharide2.residues.rescharge.interface import LibraryCharger
from .workflow import rct_protocol

# building parameters
outdir = Path('RCT_test')
outdir.mkdir(exist_ok=True)

DOP = 8
N = 100
chgr = MolCharger.subclass_registry['Espaloma-AM1-BCC']()
delete_pdb : bool=True
save_sdf : bool=True

term_groups = { 
    'peg_modified' : {
        'peg_TERM2' : 'head',
        'peg_TERM3' : 'tail',
    },
    'paam_modified' : {
        'paam_TERM2' : 'head',
        'paam_TERM3' : 'tail',
    },
    'pnipam_modified' : {
        'pnipam_TERM2' : 'head',
        'pnipam_TERM3' : 'tail',
    },
}

# defining paths
mol_category = 'simple_polymers'
pdb_dir  = Path(f'pdb_test_cleaned/pdbs/{mol_category}')
mono_dir = Path(f'pdb_test_cleaned/monos/{mol_category}')

# estimation and building loop
for mol_name, term_group_orient in term_groups.items():
    pdb_temp   = outdir / f'{mol_name}_redux.pdb' # NOTe : new PDB path, not an existing one
    # pdb_temp = 'polymer.pdb'
    mono_path = mono_dir / f'{mol_name}.json'
    assert(mono_path.exists())

    monogrp = MonomerGroup.from_file(mono_path)
    lib_chgs = rct_protocol(pdb_temp, monogrp, term_group_orient, N, charger=chgr, delete_pdb=delete_pdb, save_sdf=save_sdf)
    lib_chgs.to_file(outdir / f'{mol_name}_residue_charges.json')

    # PHASE 2: applying library charges to ful-size mol
    pdb_path = pdb_dir / f'{mol_name}.pdb' # path to the true, full-size PDB 
    offtop = Topology.from_pdb(pdb_path, _custom_substructures=monogrp.monomers, toolkit_registry=TKREGS['The RDKit'])
    was_partitioned = partition(offtop) 
    assert(was_partitioned)
    fullmol =topology.get_largest_offmol(offtop)
    
    lib_chgr = LibraryCharger(lib_chgs)
    rctmol = lib_chgr.charge_molecule(fullmol, in_place=False)
    topology.topology_to_sdf(outdir / f'{mol_name}_{lib_chgr.CHARGING_METHOD}.sdf', rctmol.to_topology())

# OpenMM I/O and simulation interfaces

## Initialize OpenMM sim + files

In [4]:
from openmm.unit import nanosecond, picosecond, femtosecond
from openmm.unit import kelvin, atmosphere, nanometer, centimeter
from openmm.unit import gram, mole, liter

from polysaccharide2.genutils.unitutils import openmm_to_openff, openff_to_openmm
from polysaccharide2.genutils.fileutils.pathutils import assemble_path
from polysaccharide2.genutils.logutils.IOHandlers import MSFHandlerFlex

from polysaccharide2.openmmtools import parameters, execution
from polysaccharide2.openmmtools.parameters import SimulationParameters

from polysaccharide2.openfftools import topology
from polysaccharide2.openfftools.omminter import openff_topology_to_openmm
from polysaccharide2.openfftools.solvation import packing, boxvectors, solvents
from polysaccharide2.openfftools.solvation.packing import pack_topology_with_solvent

from .workflow import vacuum_anneal


# input parameters
sdf_dir = Path('sdf_test')
sdf_path = sdf_dir / 'polyvinylchloride_AM1-BCC-ELF10.sdf'
postfix = 'conf_1'

box_dims = 4.0 * np.ones(3) * nanometer
density = 0.997 * gram/centimeter**3
exclusion = 1.0 * nanometer
ff_name = 'openff-2.0.0'

# define paths
save_dir = Path('openmm_test')
save_dir.mkdir(exist_ok=True)
solvent  = solvents.water_TIP3P

sim_param_path = Path('sim_param_sets')
anneal_params = SimulationParameters.from_file(sim_param_path / 'anneal_params.json')
prod_schedule = {
    'equilibration' : parameters.SimulationParameters.from_file(sim_param_path / 'equilibration_params.json'),
    'production' : parameters.SimulationParameters.from_file(sim_param_path / 'production_lite_params.json'),
}

# initialize simulation
with MSFHandlerFlex(save_dir, proc_name='sim_schedule', loggers='all') as log_handler:
    offtop = topology.topology_from_sdf(sdf_path)
    offmol = topology.get_largest_offmol(offtop)
    mol_name = offmol.name

    conf_top = vacuum_anneal(save_dir, offtop, anneal_params, forcefield=ff_name, box_vecs=box_dims)
    solv_top = pack_topology_with_solvent(conf_top, solvent, box_vecs=box_dims, density=density, exclusion=exclusion)
    solv_top_path = assemble_path(save_dir, f'{mol_name}_solv_{solvent.name}', extension='sdf', postfix=postfix)
    topology.topology_to_sdf(solv_top_path, solv_top)

    # execute remaining simulations
    ommtop, ommsys, ommpos = openff_topology_to_openmm(solv_top, forcefield=ff_name, box_vecs=box_dims)
    history = execution.run_simulation_schedule(save_dir, prod_schedule, ommtop, ommsys, ommpos, return_history=True)

# Playing with ratios

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, ClassVar, TypeVar
from math import gcd
from numbers import Number


N = TypeVar('N')
def sgnmag(num : N) -> tuple[bool, N]:
    '''Returns the sign and magnitude of a numeric-like value'''
    return num < 0, abs(num)


@dataclass(repr=False)
class Ratio:
    '''For representing fractional ratios between two objects'''
    num   : Any
    denom : Any

    # REPRESENTATION
    def __repr__(self) -> str:
        return f'{self.num}/{self.denom}'
    
    def to_latex(self) -> str:
        '''Return latex-compatible string which represent fraction'''
        return rf'\frac{{{self.num}}}{{{self.denom}}}'

    # RELATIONS
    @property
    def reciprocal(self) -> 'Ratio':
        '''Return the reciprocal of a ration'''
        return self.__class__(self.denom, self.num)


@dataclass(repr=False)
class Rational(Ratio):
    '''For representing ratios of integers'''
    num   : int
    denom : int

    # REDUCTION
    autoreduce : ClassVar[bool]=False
    
    def __post_init__(self) -> None:
        if self.__class__.autoreduce:
            self.reduce()

    def reduce(self) -> None:
        '''Reduce numerator and denominator by greatest common factor'''
        _gcd = gcd(self.num, self.denom)
        self.num=int(self.num / _gcd)
        self.denom=int(self.denom / _gcd)
    simplify = reduce # alias for convenience

    @property
    def reduced(self) -> 'Rational':
        '''Return reduced Rational equivalent to the current rational (does not modify in-place)'''
        new_rat = self.__class__(self.num, self.denom)
        new_rat.reduce()

        return new_rat
    simplifed = reduced # alias for convenience
    
    def as_proper(self) -> tuple[int, 'Rational']:
        '''Returns the integer and proper fractional component of a ratio'''
        integ, remain = divmod(self.num, self.denom)
        return integ, self.__class__(remain, self.denom)
    
    # ARITHMETIC
    def __add__(self, other : 'Rational') -> 'Rational':
        '''Sum of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) + (self.denom * other.num),
            denom=(self.denom * other.denom)
        )
    
    def __sub__(self, other : 'Rational') -> 'Rational':
        '''Difference of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) - (self.denom * other.num),
            denom=(self.denom * other.denom)
        )

    def __mul__(self, other : 'Rational') -> 'Rational':
        '''Product of two Rationals'''
        return self.__class__(
            num=self.num * other.num,
            denom=self.denom * other.denom
        )

    def __div__(self, other : 'Rational') -> 'Rational':
        '''Quotient of two Rationals'''
        return self.__class__(
            num=self.num * other.denom,
            denom=self.denom * other.num
        )
    
    def __pow__(self, power : float) -> 'Rational':
        '''Exponentiates a ratio'''
        return self.__class__(
            num=self.num**power,
            denom=self.denom**power
        )

In [ ]:
p = Rational(3, 6)
q = Rational(4, 12)

print(p, p.reciprocal, p.reduced, p+q)

In [ ]:
Rational.autoreduce = False

In [ ]:
import numpy as np
from numbers import Number

for val in (4, 4.0, 4+0j, np.pi, '4', [4], False, 'sgdfg'):
    print(val, type(val), isinstance(val, Number))

In [ ]:
from fractions import Fraction